# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
df = pd.read_csv("citydata.csv")
print(df.columns)

cities_df=pd.DataFrame()
cities_df["City"]=df["City"]
cities_df["Lat"]=df["Lat"]
cities_df["Lng"]=df["Lng"]
cities_df["Max_Temp"]=df["Max_Temp"]
cities_df["Humidity"]=df["Humidity"]
cities_df["Cloudiness"]=df["Cloudiness"]
cities_df["Wind_Speed"]=df["Wind_Speed"]
cities_df["Country"]=df["Country"]
cities_df["Date"]=df["Date"]

cities_df.head()


In [ ]:
cities_df.info()

In [ ]:
cities_df = cities_df.dropna()
cities_df.info()

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
gmaps.configure(api_key=g_key)

In [ ]:
locations = cities_df[["Lat", "Lng"]].astype(float)
weight=(cities_df["Humidity"])

fig = gmaps.figure()

heat_layer=gmaps.heatmap_layer(locations, weights=weight,dissipating=False, max_intensity=100, point_radius = 1)


fig.add_layer(heat_layer)

fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
cities_df2=cities_df.loc[cities_df["Max_Temp"]>70]
cities_df3=cities_df2.loc[cities_df2["Max_Temp"]<80]
cities_df3=cities_df2.loc[cities_df2["Wind_Speed"]<10]
ideal_df=cities_df3.loc[cities_df3["Cloudiness"]==0]
ideal_df=ideal_df.reset_index(drop=True)
ideal_df.head(10)

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
drafthotel_df=pd.DataFrame()
drafthotel_df["Hotel Name"]=()


drafthotel_df

In [ ]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "keyword": "hotel",
    "radius": 5000,
    "type": "hotel",
    "key": g_key
}

for index, row in ideal_df.iterrows():
       # get location type from df
    location = f'{row["Lat"]}, {row["Lng"]}'

    # add location to params dict
    params['location'] = location

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        
        ideal_df.loc[index, 'Hotel_Name'] = results[0]['name']
              
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    

In [ ]:
drafthotel_df["Hotel Name"]=ideal_df["Hotel_Name"]
drafthotel_df["City"]=ideal_df["City"]
drafthotel_df["Country"]=ideal_df["Country"]
drafthotel_df["Humidity"]=ideal_df["Humidity"]
drafthotel_df["Lat"]=ideal_df["Lat"]
drafthotel_df["Lng"]=ideal_df["Lng"]

hotel_df=drafthotel_df.dropna()

hotel_df.head()

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
locations = hotel_df[["Lat", "Lng"]]
weight=(hotel_df["Humidity"])

fig = gmaps.figure()

heat_layer=gmaps.heatmap_layer(locations, weights=weight)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

fig.add_layer(heat_layer)

# Display figure
fig


